In [1]:
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from matplotlib.colors import BoundaryNorm
import warnings


In [ ]:
#read in newvalue_future and _historical and sumbin_future and _historical and plot

warnings.filterwarnings('ignore', category=UserWarning)


scenarios = ["rcp26","rcp60"]
historical_time= 1146
future_time=65

years = ['1845', '1990', '1995', '2009', '2010', '2020', '2026', '2032', '2048', '2050','2052', '2056', '2080', '2100', '2150', '2200', '2250']
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(30, 15), subplot_kw={'projection': ccrs.PlateCarree()}, layout="compressed")
cmap = matplotlib.colors.ListedColormap(['white', 'green'])
countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

plot_idx = 0

#sub_africa = sub_africa.rename({'latitude': 'lat', 'longitude': 'lon'})

for row, scenario in enumerate(scenarios):

    newvalue_future = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CC_{future_time}_{scenario}.nc", decode_times=False).to_array().isel(variable=0)
    sumbin_future = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CCLUC_{future_time}_{scenario}.nc", decode_times=False).to_array().isel(time=0)
    sumbin_future= sumbin_future.isel(variable=0)

    newvalue_hist = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CC_{historical_time}_{scenario}.nc", decode_times=False).to_array().isel(variable=0)
    sumbin_hist = xr.open_dataset(f"/storage/scratch/users/ch21o450/data/SR/SR_CCLUC_{historical_time}_{scenario}.nc", decode_times=False).to_array().isel(variable=0)
    



    diff_newvalue = newvalue_future - newvalue_hist
    diff_sumbin = sumbin_future - sumbin_hist
    
    if plot_idx >= len(axes.flatten()):
        break
    ax1 = axes.flatten()[plot_idx]
    ax2 = axes.flatten()[plot_idx + 1]

    labels = ['a', 'b']
    # Define the colormap and the range of values for the colorbar

    #vmin_b = diff_sum_bin.min().values
    #vmax_b = diff_sum_bin.max().values

    cmap = plt.colormaps['RdYlGn']
    boundaries = [-200, -150, -100, -50,-5,5,50, 100, 150, 200] # Adjust these values according to your data
    norm = BoundaryNorm(boundaries, cmap.N)

    # Plot the ensemble mean difference for each panel
    im1 = ax1.pcolormesh(diff_newvalue['lon'].values, diff_newvalue['lat'].values, np.where(diff_newvalue.values != 0, diff_newvalue.values, np.nan), transform=ccrs.PlateCarree(), cmap=cmap, norm=norm)

    countries.plot(ax=ax1, color="lightgrey", zorder=1, alpha=0.3)
    ax1.set_title(f"Climate Change impact: {year_indices[future_time]} - 1995 for {scenario}",loc='center')

    ax1.axis('off')
    ax1.set_extent((-180,180,-63,90))
    ax1.add_feature(cfeature.BORDERS,color='white',linewidth=0.5)

    ticks_b = np.array( [-200, -150, -100, -50,0,50, 100, 150, 200])
    tick_labels = [str(int(tick)) for tick in ticks_b]

    cbar1 = plt.colorbar(im1, ax=ax1, fraction=0.024, pad=0.04, spacing='proportional', ticks=ticks_b,extend='both')
    cbar1.set_ticklabels(tick_labels)

    im2 = ax2.pcolormesh(diff_sumbin['lon'].values, diff_sumbin['lat'].values,  np.where(diff_sumbin.values != 0, diff_sumbin.values, np.nan), transform=ccrs.PlateCarree(), cmap=cmap, norm=norm)

    countries.plot(ax=ax2, color="lightgrey", zorder=1, alpha=0.3)

    ax2.set_title(f"Climate Change + Land-Use Change impact: {year_indices[future_time]} - 1995 for {scenario}",loc='center')

    ax2.axis('off')
    ax2.set_extent((-180,180,-63,90))
    cbar2 = plt.colorbar(im2, ax=ax2, fraction=0.024, pad=0.04, spacing='proportional', ticks=ticks_b,extend='both')
    cbar2.set_ticklabels(tick_labels)

    # Increase the plot index by 2 to move to the next triplet of subplots
    plot_idx += 2

#plt.suptitle(future_time , size=16, y=0.8)
#plt.savefig("/storage/homefs/ch21o450/scripts/BioScenComb/main_figures/Figure_2")

plt.show() 
